In [167]:
import rdflib
from rdflib import Graph
from rdflib.namespace import DC, RDF, FOAF, RDFS, TIME
from rdflib import Namespace
from rdflib import URIRef, BNode, Literal
from typing import List

# 1. Property Graph vs RDF/SPARQL

## 1. Create an RDF Graph representing the same road network and travel times

In [4]:
# Grab the definitions from https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Graphs_Queries.ipynb#scrollTo=TSYQfoXu3b_l
cities = [
    "Inverness",
    "Aberdeen",
    "Leuchars",
    "StAndrews",
    "Edinburgh",
    "Glasgow",
    "York",
    "Cologne",
    "Carlisle",
    "Birmingham",
    "London",
    "Brussels",
    "Toronto",
    "Winnipeg",
    "Saskatoon",
    "Edmonton",
    "Jasper",
    "Vancouver"
  ];

connections = [
    ( "Inverness", "Aberdeen", 3, 2.5 ),
    ( "Aberdeen", "Leuchars", 1.5, 1 ),
    ( "Leuchars", "Edinburgh", 1.5, 3 ),
    ( "Edinburgh", "Glasgow", 1, 1 ),
    ( "Edinburgh", "York", 3.5, 4 ),
    ( "Glasgow", "Carlisle", 1, 1 ),
    ( "Carlisle", "York", 2.5, 3.5 ),
    ( "Carlisle", "Birmingham", 2.0, 1 ),
    ( "Birmingham", "London", 1.5, 2.5 ),
    ( "Leuchars", "StAndrews", 0.2, 0.2 ),
    ( "York", "London", 1.8, 2.0 ),
    ( "London", "Brussels", 2.5, 3.5 ),
    ( "Brussels", "Cologne", 2, 1.5 ),
    ( "Toronto", "Winnipeg", 36, 35 ),
    ( "Winnipeg", "Saskatoon", 12, 5 ),
    ( "Saskatoon", "Edmonton", 12, 17 ),
    ( "Edmonton", "Jasper", 6, 5 ),
    ( "Jasper", "Vancouver", 12, 13 )
]


In [158]:
# We were unable to find the predicates in the available namespaces; define the needed relations
base_url = "http://example.org/"
n = Namespace(base_url)
Trip = n.Trip
departing = rdflib.term.URIRef(base_url + "departing")
arriving = rdflib.term.URIRef(base_url + "arriving")
reachable = rdflib.term.URIRef(base_url + "reachable")

In [159]:
# Parse the data as triplets
g = Graph()

nodes = { n: BNode() for n in cities}

for name,node in nodes.items():
    g.add((node, RDF.type, n.City))
    g.add((node, FOAF.name, Literal(name)))

for city1, city2, time1, time2 in connections:    
    trip1 = BNode()
    trip2 = BNode()
    
    g.add((trip1, RDF.type, Trip))
    g.add((trip2, RDF.type, Trip))
    
    g.add((trip1, TIME.duration, Literal(time1)))
    g.add((trip2, TIME.duration, Literal(time2)))
    
    g.add((nodes[city1], departing, trip1))
    g.add((nodes[city2], arriving, trip1))
    
    g.add((nodes[city1], departing, trip2))
    g.add((nodes[city2], arriving, trip2))
    
    g.add((nodes[city1], reachable, nodes[city2]))
    g.add((nodes[city2], reachable, nodes[city1]))

# Print all triples
# for s, p, o in g:
#     print((s, p, o))

## 2. Implement a SPARQL query returning all cities which can be reached from London. 

In [175]:
# Multi-hop neighbours

result = g.query(
    """SELECT DISTINCT ?name
    WHERE
    {
        ?city1 foaf:name "London" .
        ?city1 ex:reachable+ ?city2.
        ?city2 foaf:name ?name
    }""", initNs={ 'foaf': FOAF,  'ex' : base_url })

for row in result:
    print(row)

(rdflib.term.Literal('Brussels'),)
(rdflib.term.Literal('Cologne'),)
(rdflib.term.Literal('London'),)
(rdflib.term.Literal('York'),)
(rdflib.term.Literal('Carlisle'),)
(rdflib.term.Literal('Glasgow'),)
(rdflib.term.Literal('Edinburgh'),)
(rdflib.term.Literal('Leuchars'),)
(rdflib.term.Literal('Aberdeen'),)
(rdflib.term.Literal('Inverness'),)
(rdflib.term.Literal('StAndrews'),)
(rdflib.term.Literal('Birmingham'),)


### Bonus: all cities which can be reached within less than 5 hours. Hint: You might want to consider property paths.

In [166]:
result = g.query(
""" SELECT DISTINCT ?name
WHERE
{
    PATH ?p FROM ?city1 TO ?city2 {
     ?r a :Trip ;
        :departing ?city1 ;
        :arriving ?city2 ;
        :duration ?dur
    }
    ORDER BY sum(project(?p, ?distance))
    LIMIT 1
}
""", initNs={ 'foaf': FOAF, 'rdf' : RDF, 'ex' : base_url })

for row in result:
    print(row)

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found 'P'  (at char 35), (line:4, col:5)

## 3. Implement generic python code (i.e., the algorithms don’t have to be specified in SPARQL, but could be) for the Single Source Shortest Path algorithm and return the shortest paths to all other cities starting from London. You can choose either Dijkstra’s or Bellman-Ford’s algorithm.

In [235]:
def neighbour(name, g) -> List[str]:
    result = g.query(
    """SELECT ?name ?dur
    WHERE
    {
        ?city1 foaf:name "%s" .
        ?city2 foaf:name ?name .
        {
            ?city1 ex:departing ?trip .
            ?city2 ex:arriving ?trip .
        }
        UNION
        {
            ?city2 ex:departing ?trip .
            ?city1 ex:arriving ?trip .
        }
        ?trip time:duration ?dur .
    }""" % name, initNs={ 'foaf': FOAF, 'time': TIME,  'ex' : base_url })
    return [
        (
            r[0].value,
#             r[1].value
        ) 
        for r in result
    ]

neighbour("London", g)

[('Brussels',),
 ('Brussels',),
 ('Birmingham',),
 ('Birmingham',),
 ('York',),
 ('York',)]

In [182]:
"""SELECT DISTINCT ?name ?dur
    WHERE
    {
        ?city1 foaf:name %s .
        ?city1 ex:reachable ?city2.
        ?city2 foaf:name ?name
        ?city1 foaf:name ?name
    }""" % name

'SELECT DISTINCT ?name ?dur\n    WHERE\n    {\n        ?city1 foaf:name Vancouver .\n        ?city1 ex:reachable ?city2.\n        ?city2 foaf:name ?name\n        ?city1 foaf:name ?name\n    }'